In [3]:
# app.py
#pip install langchain chromadb sentence-transformers
!pip install streamlit
import streamlit as st
import os
import uuid
import fitz  # PyMuPDF
import pdfplumber
import faiss
import numpy as np




  Using cached streamlit-1.40.1-py2.py3-none-any.whl (8.6 MB)
  Using cached altair-5.4.1-py3-none-any.whl (658 kB)
  Using cached watchdog-4.0.2-py3-none-win_amd64.whl (82 kB)
  Using cached blinker-1.8.2-py3-none-any.whl (9.5 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl (6.9 MB)
  Using cached gitpython-3.1.45-py3-none-any.whl (208 kB)
  Using cached pyarrow-17.0.0-cp38-cp38-win_amd64.whl (25.2 MB)
  Using cached rich-13.9.4-py3-none-any.whl (242 kB)
  Using cached narwhals-1.42.1-py3-none-any.whl (359 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl (62 kB)
  Using cached smmap-5.0.2-py3-none-any.whl (24 kB)
  Attempting uninstall: watchdog
    Found existing installation: watchdog 1.0.2
    Uninstalling watchdog-1.0.2:
      Successfully uninstalled watchdog-1.0.2
  Attempting uninstall: rich
    Found existing installation: rich 14.2.0
    Uninstalling rich-14.2.0:
      Successfully uninstalled rich-14.2.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.2.5 requires pyqt5<5.13, which is not installed.
spyder 4.2.5 requires pyqtwebengine<5.13, which is not installed.
spyder 4.2.5 requires watchdog<2.0.0,>=0.10.3, but you have watchdog 4.0.2 which is incompatible.


ModuleNotFoundError: No module named 'fitz'

In [10]:
from sentence_transformers import SentenceTransformer
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFaceHub

In [4]:
uploaded_file = st.file_uploader("Upload a PDF", type="pdf")

if uploaded_file:
    session_id = st.session_state.get("session_id", str(uuid.uuid4()))
    st.session_state.session_id = session_id

    pdf_path = f"uploaded_{session_id}.pdf"
    with open(pdf_path, "wb") as f:
        f.write(uploaded_file.getbuffer())


2025-12-30 16:22:58.667 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-30 16:22:58.669 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-30 16:22:58.672 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-30 16:22:58.682 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-30 16:22:58.844 
  command:

    streamlit run c:\users\sandr\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-12-30 16:22:58.846 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [5]:
def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

def extract_tables_from_pdf(pdf_path):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            table = page.extract_table()
            if table:
                tables.append("\n".join(["\t".join(row) for row in table]))
    return tables


In [6]:
def create_chunks(text, chunk_size=500):
    chunks = []
    current = ""
    for line in text.split("\n"):
        current += line + " "
        if len(current) > chunk_size:
            chunks.append(current)
            current = ""
    if current:
        chunks.append(current)
    return chunks




In [8]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

def create_faiss_index(chunks):
    embeddings = np.array([embedding_model.encode(c) for c in chunks])
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return index


c:\users\sandr\anaconda3\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [12]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "ENTER_YOUR_KEY"


In [15]:
llm = HuggingFaceHub(
    repo_id="google/flan-t5-base",
    task="text2text-generation",
    model_kwargs={"temperature": 0.3, "max_new_tokens": 256}
)



In [17]:
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Answer the question based on the context below.

Context:
{context}

Question:
{question}

Answer:
"""
)

qa_chain = LLMChain(llm=llm, prompt=prompt)




C:\Users\sandr\AppData\Local\Temp\ipykernel_14060\1271036605.py:16: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  qa_chain = LLMChain(llm=llm, prompt=prompt)


In [20]:
# -----------------------------
# Q&A Section
# -----------------------------
question = st.text_input("Ask a question about the PDF:")

if question and st.session_state.vectorstore:
    retriever = st.session_state.vectorstore.as_retriever(search_kwargs={"k": 3})
    docs = retriever.get_relevant_documents(question)
    context = "\n".join([doc.page_content for doc in docs])

    answer = qa_chain.run(
        {"context": context, "question": question}
    )

    st.subheader("Answer")
    st.write(answer)




2025-12-30 20:10:38.804 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-30 20:10:38.825 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-30 20:10:38.832 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-30 20:10:38.844 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-30 20:10:38.849 Session state does not function when running a script without `streamlit run`
2025-12-30 20:10:38.887 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-30 20:10:38.892 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
